# 毕业论文`GRAD`
## 计算贡献值

---
*@author: Evan*\
*@date: 2023-11-14*

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd

import sys
sys.path.append('../../src/')
from mask import polygon_to_mask
from namelist import *

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

导出模拟、观测与模拟试验的各年份的O3平均值

In [4]:
simvar = 'O3'
obsvar = 'O3'

In [5]:
df = pd.DataFrame(
    index=pd.date_range(start='2023-09-01T00', end='2023-09-30T23', freq='H')
)
years = [2014,2015,2016,2019,2021,2022]

for year in years:
    ds = xr.open_dataset(datadir + f'processed/Sep_{year}/Sep_{year}_chem.nc')
    data_sim = ds[simvar][:,0,:,:]

    shp = gpd.read_file(shp_PRD_adm)
    lon = data_sim.longitude
    lat = data_sim.latitude
    mask    = polygon_to_mask(shp.geometry[0], lon, lat)
    mask_da = xr.DataArray(mask, dims=('y','x'))
    masked_sim  = data_sim.where(mask_da)
    sim    = masked_sim.mean(dim=('x','y'),skipna=True)

    dfobs = pd.read_excel(obsSep + f'site_{obsvar}_{year}.xlsx',index_col=0)
    obs = dfobs.mean(axis=1,skipna=True)
    obs.interpolate(method='linear',inplace=True)
    
    df[f'sim_{year}']=sim
    df[f'obs_{year}']=obs.values
    print(f'Complete {year}')

for year in [2019,2021,2022]:
    ds = xr.open_dataset(datadir + f'processed/Annually_Sep_{year}/Sep_{year}_chem.nc')
    data_sim = ds[simvar][:,0,:,:]

    shp = gpd.read_file(shp_PRD_adm)
    lon = data_sim.longitude
    lat = data_sim.latitude
    mask    = polygon_to_mask(shp.geometry[0], lon, lat)
    mask_da = xr.DataArray(mask, dims=('y','x'))
    masked_sim  = data_sim.where(mask_da)
    sim    = masked_sim.mean(dim=('x','y'),skipna=True)
    df[f'sen_{year}']=sim
    print(f'Complete Sens {year}')

Complete 2014
Complete 2015
Complete 2016
Complete 2019
Complete 2021
Complete 2022
Complete 2019
Complete 2021
Complete 2022


In [7]:
df.to_excel(datadir + 'Contribution/contribution_hours.xlsx',index=True)